In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [18]:
df = pd.read_csv('cleaned_income_evaluation.csv')
df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## 1. Modeling (Log Regression)

### 1.1. Feature engineering

In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

# Separate features and target
X = df.drop('income', axis=1)
y = LabelEncoder().fit_transform(df['income'])  # Encoding the target

# Identify column types
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Preprocessing for numerical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # optional
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

# Combine preprocessors
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


### 1.2. Model execution

In [20]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [21]:
from sklearn.ensemble import RandomForestClassifier

# Full pipeline with model
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(max_iter=1000))
])

# Fit model
model_pipeline.fit(X_train, y_train)
y_pred = model_pipeline.predict(X_test)

In [23]:
import pickle

with open("log-regression.pkl", "wb") as file:
    pickle.dump(model_pipeline, file)

with open("log-reg_preprocessor.pkl", "wb") as file:
    pickle.dump(preprocessor, file)
    
print(f"Best model saved")

Best model saved


In [22]:
# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
classification_report_result = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Create a summary DataFrame for metrics
results = {
    'Metric': ['Accuracy'],
    'Value': [accuracy]
}
results_df = pd.DataFrame(results)

print("=== Logistic Regression Model Results ===\n")
print(results_df.to_string(index=False))

print("\n=== Detailed Classification Report ===\n")
print(classification_report_result)

print("=== Confusion Matrix ===\n")
conf_matrix_df = pd.DataFrame(conf_matrix, 
                              index=["Actual ≤50K", "Actual >50K"], 
                              columns=["Predicted ≤50K", "Predicted >50K"])
print(conf_matrix_df)


=== Logistic Regression Model Results ===

  Metric    Value
Accuracy 0.847755

=== Detailed Classification Report ===

              precision    recall  f1-score   support

           0       0.88      0.93      0.90      4852
           1       0.73      0.58      0.65      1539

    accuracy                           0.85      6391
   macro avg       0.80      0.76      0.78      6391
weighted avg       0.84      0.85      0.84      6391

=== Confusion Matrix ===

             Predicted ≤50K  Predicted >50K
Actual ≤50K            4519             333
Actual >50K             640             899
